In [163]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv

load_dotenv()

api_key_4square = os.getenv('FOURSQUARE_KEY')
api_key_yelp = os.getenv('YELP_KEY')

In [177]:
# import bikes_df
bikes_df =pd.read_csv(r"B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\bikes_df.csv",sep=',')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [165]:
# 4square api prep function
def places_4square(latitude, longitude, radius, categories,limit, api_key):
    params = {}
    ll = str(latitude)+","+str(longitude)
    params["ll"] = ll
    params['radius'] = radius
    categories = ','.join(str(x)for x in categories)
    params["categories"] = categories
    params["limit"] = limit

    url = "https://api.foursquare.com/v3/places/search"
    headers = {"accept": "application/json","Authorization": api_key}
    response = requests.get(url, params=params, headers=headers)
    return response

In [166]:
# # Getting and saving response data as a local json file
# category_list=[16033,13003,13029]
# foursquare_places_json = places_4square(bikes_df['latitude'][0],bikes_df['longitude'][0],1000,category_list,50,api_key_4square).json()
# with open('data/foursquare_places_data.json','w') as f:
#     json.dump(foursquare_places_json,f)
# foursquare_places_json

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [167]:
# pulling data from json into dataframe
# lists for temp busckets
id = []
categories = []
distance = []
name = []
bar = []
dog_park = []
# lists to normalize category list
bar_lookup = ['bar', 'pub', 'speakeasy', 'beer', 'Brewery']
dog_park_lookup = ['dog park']


#loading json file
with open(r"B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\foursquare_places_data.json",'r') as f:
    data= json.load(f)
    
# run through all results for each place info
for result in range(len(data['results'])):
    id.append(data['results'][result]['fsq_id'])
    # run thorugh all categories for each place
    for i in range(len(data['results'][result]['categories'])):
        if any(map(data['results'][result]['categories'][i]['short_name'].lower().__contains__, bar_lookup)):
            bar.append(1)
            dog_park.append(0)
            break

        elif any(map(data['results'][result]['categories'][i]['short_name'].lower().__contains__, dog_park_lookup)):
            bar.append(0)
            dog_park.append(1)
            break
        else:
            bar.append(9)
            dog_park.append(9)
            break
        
    distance.append(data['results'][result]['distance'])
    name.append(data['results'][result]['name'])

Put your parsed results into a DataFrame

In [168]:
# dataframe creation
foursquare_places_df = pd.DataFrame({'id':id, 'name':name,"bar":bar,'dog_park': dog_park,'distance':distance})
foursquare_places_df

,id,name,bar,dog_park,distance
0,4e1c4c481fc714dc5c3143ed,BrewDog Glasgow,1,0,884
1,4b76fff5f964a52018732ee3,The Lismore,1,0,636
2,50d20bc9e4b029fc6804c7a9,The Sparkle Horse,1,0,705
3,4d0263ce3fd4224b72abb677,Three Judges,1,0,632
4,57cf101c498e00f91f9e64cc,Bag O Nails,1,0,636
5,56db6679498e2749a7e2fffe,Six Degrees North,1,0,839
6,4ee28df99a521fcbefc9cf9a,Dukes Bar,1,0,831
7,51699468e4b02c13d46e0511,The Record Factory,9,9,692
8,5132030ae4b038fff6d6dda1,Bar Gumbo,1,0,833
9,89a280b505ec43694e54bea7,Obsession Club Glasgow,1,0,371


In [169]:
# filter data to just bars/pubs and dog parks
foursquare_places_df = foursquare_places_df.query('bar <= 1 & dog_park <=1')
foursquare_places_df

,id,name,bar,dog_park,distance
0,4e1c4c481fc714dc5c3143ed,BrewDog Glasgow,1,0,884
1,4b76fff5f964a52018732ee3,The Lismore,1,0,636
2,50d20bc9e4b029fc6804c7a9,The Sparkle Horse,1,0,705
3,4d0263ce3fd4224b72abb677,Three Judges,1,0,632
4,57cf101c498e00f91f9e64cc,Bag O Nails,1,0,636
5,56db6679498e2749a7e2fffe,Six Degrees North,1,0,839
6,4ee28df99a521fcbefc9cf9a,Dukes Bar,1,0,831
8,5132030ae4b038fff6d6dda1,Bar Gumbo,1,0,833
9,89a280b505ec43694e54bea7,Obsession Club Glasgow,1,0,371
10,4d94c74fb550a0939f928e87,Smiddy Bar,1,0,571


In [170]:
# save dataframe as CSV to access in other notebooks
foursquare_places_df.to_csv(r'B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\foursquare_places_df.csv',sep =',',index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [171]:
# yelp api prep function
def places_yelp(latitude, longitude, radius, categories,limit, api_key):
    params = {}
    params["longitude"] = longitude
    params["latitude"] = latitude
    params['radius'] = radius
    params["categories"] = categories
    params["limit"] = limit

    url = f"https://api.yelp.com/v3/businesses/search?"
    headers = {"accept": "application/json","Authorization": f'bearer {api_key}'}
    response = requests.get(url, params=params, headers=headers)
    return response

In [172]:
# # Getting and saving response data as a local json file
# category_list=['bars', 'dog_parks']
# yelp_places_json = places_yelp(bikes_df['latitude'][0],bikes_df['longitude'][0],1000,category_list,10,api_key_yelp).json()
# with open('data/yelp_places_data.json','w') as f:
#     json.dump(yelp_places_json,f)
# yelp_places_json

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [173]:
# pulling data from json into dataframe
# lists for temp busckets
id = []
categories = []
distance = []
name = []


#loading json file
with open(r"B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\yelp_places_data.json",'r') as f:
    data= json.load(f)
    
# run through all results for each place info
for result in range(len(data['businesses'])):
    id.append(data['businesses'][result]['id'])
    categories.append(data['businesses'][result]['categories'][0]['alias'])
    distance.append(data['businesses'][result]['distance'])
    name.append(data['businesses'][result]['name'])

Put your parsed results into a DataFrame

In [174]:
# dataframe creation
yelp_places_df = pd.DataFrame({'id':id, 'name':name,"categories":categories,'distance':distance})
yelp_places_df

,id,name,categories,distance
0,qjjmYSR3CVsJlr75vyIEBg,BrewDog Glasgow,bars,871.079804
1,PkLiavkwqsGzE5D2tqfsyg,The Sparkle Horse,pubs,689.080576
2,58dUYvYmmE2h19Q3qITHaA,Brel,belgian,1237.724147
3,_S0eZCB0-qNCAMTrlAIqCw,6 Degrees North,pubs,854.560573
4,Gi2Fs1Ve7lyJAXWKeQkthg,Three Judges,pubs,637.835306
5,c2fwMshDF0wtCKGLe3IGtw,Dukes,pubs,838.518390
6,NTklMMCjRBKMklSwl4YvXg,Tennents Bar,pubs,1141.982966
7,V3ma6UVAs0FWx9eHHSnnMQ,Innis & Gunn Beer Kitchen,beerbar,1240.339614
8,dMdrLM-SMYuiGzzo6Si2Cw,Firebird,italian,937.443536
9,Se9BxMKBLsjTweiYc8-xJg,Vodka Wodka,pubs,1264.736501


In [175]:
# filter data to justt bars/pubs and dog parks

In [176]:
# save dataframe as CSV to access in other notebooks
yelp_places_df.to_csv(r'B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\yelp_places_df.csv',sep =',',index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating